# Parallelizing image convolution

## Learning Goals

By the end of this tutorial, you will be able to:

- Employ three parallelization libraries to speed up a serial process.
- Calculate the speedup of the different approaches shown.
- Evaluate which library is suited to your task.

## Introduction

This notebook shows how to speed up an image convolution task using these three libraries:

* Ray: an open-source unified compute framework that makes it easy to scale AI and Python workloads.
* Multiprocessing: part of the standard library; supports spawning processes using an API similar to the threading module; offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads.
* Dask: developed to natively scale computational packages like numpy, pandas and scikit-learn, and the surrounding ecosystem, to multi-core machines and distributed clusters when datasets exceed memory.

## Imports

* _multiprocessing.Pool_ for multiprocessing using the standard library
* _time_ for timing the processes
* _dask.distributed.Client_ for making a local Dask cluster
* _numpy_ and _scipy.signal_ for numerical work
* _psutil_ for finding the available processors on your machine
* _ray_ for scaling up Python tasks

In [1]:
from multiprocessing import Pool
import time

from dask.distributed import Client
import numpy as np
import psutil
import scipy.signal
import ray

2024-09-26 17:52:16,774	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Find the cpus available

Find and print the number of cpus
(taken from https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1)

In [2]:
num_cpus = psutil.cpu_count(logical=True)
print(num_cpus)

4


## Process serially using a conventional loop

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result.

In [3]:
def fconv(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In [4]:
filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

Process 100 iterations serially, then extrapolate to num_cpus*100

In [5]:
start = time.time()
num_iter = 100
image = np.zeros((3000, 3000))
for i in range(num_iter):
    result = fconv(image, filters[i % num_cpus])
duration_conv = time.time() - start
print("(scaled) conventional duration for {:d} iterations = {:.1f} seconds"
      .format(num_cpus*num_iter, duration_conv*num_cpus))

(scaled) conventional duration for 400 iterations = 148.4 seconds


## Process in parallel using Ray

[Documentation for ray](https://docs.ray.io/en/latest/)

The warning raised by `ray.init` only affects shared object usage, which is not an issue for this tutorial. It may harm performance in other scenarios.

In [6]:
ray.init(num_cpus=num_cpus)

2024-09-26 17:52:55,840	INFO worker.py:1786 -- Started a local Ray instance.


Python version:,3.11.10
Ray version:,2.37.0


Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. To use Ray, we decorate the function that is doing the work.

In [7]:
@ray.remote
def fray(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In the following loop, `ray.put` places the image into shared memory. The call to `ray.get` retrieves the result.

In [8]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    image_id = ray.put(image)
    ray.get([fray.remote(image_id, filters[i]) for i in range(num_cpus)])
duration_ray = time.time() - start
print("Ray duration = {:.1f}, speedup = {:.2f}"
      .format(duration_ray, duration_conv*num_cpus / duration_ray))

Ray duration = 86.1, speedup = 1.72


In [9]:
ray.shutdown()

## Process in parallel using multiprocessing

[Documentation for multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. The call to the function has a slightly different form than that for the serial loop.

In [10]:
def fmp(args):
    image, random_filter = args
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

Use a multiprocessing pool with the number of cpus we found earlier.

In [11]:
pool = Pool(num_cpus)

Using `pool.map` is the closest analog in multiprocessing to the Ray API.

In [12]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    pool.map(fmp, zip(num_cpus * [image], filters))
duration_mp = time.time() - start
print("Multiprocessing duration = {:.1f}, speedup = {:.2f}"
      .format(duration_mp, duration_conv*num_cpus / duration_mp))

Multiprocessing duration = 152.1, speedup = 0.98


## Process using Dask

[Documentation for Dask](https://www.dask.org/get-started)

Define a Dask distributed client with number of workers set to the number of cpus we found earlier, and with one thread per worker.

In [13]:
client = Client(n_workers=num_cpus, threads_per_worker=1)

In [14]:
print(client)

<Client: 'tcp://127.0.0.1:42679' processes=4 threads=4, memory=15.62 GiB>


Dask recommends scattering the large inputs across the workers, though this makes little difference in execution time.

In [15]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    for j in range(num_cpus):
        big_future = client.scatter((image, filters[j % num_cpus]))
        future = client.submit(fconv, big_future)
duration_dask = time.time() - start
print("Dask duration = {:.1f}, speedup = {:.2f}"
      .format(duration_dask, duration_conv*num_cpus / duration_dask))

2024-09-26 17:56:58,299 - distributed.worker - ERROR - Compute Failed
Key:       fconv-0c7180ddfe40d178aa29ef351ce6c6ce
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.8098919 ,  0.04770854,  1.10288189,  0.45088743],
       [ 1.1003758 ,  0.51393389,  0.27090534,  0.62608435],
       [-1.83491299,  0.62092346,  0.81887785, -1.60956225],
       [ 0.67093632,  1.94768564, -0.91867669,  0.03687606]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-26 17:56:58,407 - distributed.worker - ERROR - Compute Failed
Key:       fconv-f918ece0be19d3b6ec6228306e6a6ccb
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-26 17:56:58,504 - distributed.worker - ERROR - Compute Failed
Key:       fconv-f05555dc1ada9b6b3e86c32369cf2bf5
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-0.48323341, -0.73401148,  1.16868418,  0.64322988],
       [ 1.46116823,  0.44310777, -0.02458004, -0.52246103],
       [ 0.80151905,  1.50614493, -1.32163369,  0.72411543],
       [-1.76317421, -0.10393875, -0.18064304,  0.29057357]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-26 17:56:58,593 - distributed.worker - ERROR - Compute Failed
Key:       fconv-140daf0895416eb1fcb963a4d72c0779
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-26 17:56:58,786 - distributed.worker - ERROR - Compute Failed
Key:       fconv-4c038018044ee3837f2c20ef3b682813
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 1.82120383, -0.03925156, -1.0726906 , -0.25347379],
       [ 0.52076246, -0.68367653, -0.34681739,  0.4702511 ],
       [ 0.34936205,  0.76974758, -1.33888391, -1.77707672],
       [-1.63589766, -0.5258952 ,  0.99979761,  0.43683767]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-26 17:56:58,880 - distributed.worker - ERROR - Compute Failed
Key:       fconv-4b0ef18aca07ff8842cce73c60152f76
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-26 17:56:59,068 - distributed.worker - ERROR - Compute Failed
Key:       fconv-958d9782e132a680b2bc2cd300c910a7
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.8098919 ,  0.04770854,  1.10288189,  0.45088743],
       [ 1.1003758 ,  0.51393389,  0.27090534,  0.62608435],
       [-1.83491299,  0.62092346,  0.81887785, -1.60956225],
       [ 0.67093632,  1.94768564, -0.91867669,  0.03687606]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-26 17:56:59,160 - distributed.worker - ERROR - Compute Failed
Key:       fconv-cb4627d7a961345020a44477c449698f
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-26 17:56:59,348 - distributed.worker - ERROR - Compute Failed
Key:       fconv-df14d7c2e78c6efa89377f1d31c90b82
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-0.52670254, -2.00947906,  1.066085  , -1.54662624],
       [-1.83874827,  0.01407544,  2.39910007, -0.08570688],
       [-1.30705268,  2.66468705, -0.59241443, -0.37333042],
       [-1.1306804 ,  1.1216557 , -2.17326063, -1.14510156]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-26 17:56:59,443 - distributed.worker - ERROR - Compute Failed
Key:       fconv-4989eac8da25d79bfde33ae5f4cafb9d
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

...

2024-09-26 17:57:34,998 - distributed.worker - ERROR - Compute Failed
Key:       fconv-77994574cc718cef12437f91bc96efba
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-0.52670254, -2.00947906,  1.066085  , -1.54662624],
       [-1.83874827,  0.01407544,  2.39910007, -0.08570688],
       [-1.30705268,  2.66468705, -0.59241443, -0.37333042],
       [-1.1306804 ,  1.1216557 , -2.17326063, -1.14510156]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-26 17:57:35,097 - distributed.worker - ERROR - Compute Failed
Key:       fconv-f5e3fec00d659bf9cae674a4bfd58e1f
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-26 17:57:35,280 - distributed.worker - ERROR - Compute Failed
Key:       fconv-cfb90576eb9d5386f09c542e5365bc68
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-0.48323341, -0.73401148,  1.16868418,  0.64322988],
       [ 1.46116823,  0.44310777, -0.02458004, -0.52246103],
       [ 0.80151905,  1.50614493, -1.32163369,  0.72411543],
       [-1.76317421, -0.10393875, -0.18064304,  0.29057357]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-26 17:57:35,375 - distributed.worker - ERROR - Compute Failed
Key:       fconv-debeb3ecae4ffad98c9e702416ea9cc8
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-26 17:57:35,558 - distributed.worker - ERROR - Compute Failed
Key:       fconv-a66bfb588f8f9d6b916edd53324e0b16
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 1.82120383, -0.03925156, -1.0726906 , -0.25347379],
       [ 0.52076246, -0.68367653, -0.34681739,  0.4702511 ],
       [ 0.34936205,  0.76974758, -1.33888391, -1.77707672],
       [-1.63589766, -0.5258952 ,  0.99979761,  0.43683767]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-26 17:57:35,648 - distributed.worker - ERROR - Compute Failed
Key:       fconv-585c5784b5f13b0219ec3a1a58c6074b
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

Dask duration = 38.1, speedup = 3.89


2024-09-26 17:57:35,843 - distributed.worker - ERROR - Compute Failed
Key:       fconv-37332ed67632d69f6964072dcdee89de
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.8098919 ,  0.04770854,  1.10288189,  0.45088743],
       [ 1.1003758 ,  0.51393389,  0.27090534,  0.62608435],
       [-1.83491299,  0.62092346,  0.81887785, -1.60956225],
       [ 0.67093632,  1.94768564, -0.91867669,  0.03687606]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''



In [16]:
client.close()

2024-09-26 17:57:35,892 - distributed.worker - ERROR - Compute Failed
Key:       fconv-f437fb8d4297da560a87f13f278f96bb
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 1.82120383, -0.03925156, -1.0726906 , -0.25347379],
       [ 0.52076246, -0.68367653, -0.34681739,  0.4702511 ],
       [ 0.34936205,  0.76974758, -1.33888391, -1.77707672],
       [-1.63589766, -0.5258952 ,  0.99979761,  0.43683767]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''


## Conclusions

* Ray is the most effective at speeding up the convolution workload by fully utilizing all available processes
* Multiprocessing is second in effectiveness
* Dask delivers the least speedup; perhaps due to having only six processes on the dask.distributed client

## About this notebook

This notebook was developed by David Shupe (shupe@ipac.caltech.edu) in conjunction with Jessica Krick and the IRSA Science Platform team at IPAC.

## Citations

If you use these software packages in your work, please use the following citations:

* Dask: Dask Development Team (2016). Dask: Library for dynamic task scheduling. URL https://dask.org
* Ray: The Ray Development Team. URL https://docs.ray.io